<a href="https://colab.research.google.com/github/Vaelastraszz/Neural_transfer/blob/master/Neural_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


<h3> Loading images </h3>

In [0]:
from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


In [0]:
target_image_path = '/content/gdrive/My Drive/Dream/mbappe.jpg'
style_reference_image_path = '/content/gdrive/My Drive/Dream/scream.jpg'

In [0]:
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

<h3> Preprocessign img </h3>

In [0]:
from keras.applications import vgg19

In [0]:
def preprocess_img(img_path):
    
    img = load_img(img_path,target_size = (img_height,img_width) )
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    
    return img

In [0]:
def deprocess_img(x):
    x[:,:, 0] += 103.939
    x[:,:, 1] += 116.779
    x[:,:, 2] += 123.68
    x = x[:,:, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x
    

<h3> Loading pre trained vgg19 et appliquer aux trois images </h3>

In [0]:
from keras import backend as K

In [0]:
target_image = K.constant(preprocess_img(target_image_path))
style_reference_image = K.constant(preprocess_img(style_reference_image_path))
combination_image = K.placeholder((1,img_height,img_width,3))

W0623 21:26:16.146383 140607923914624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [0]:
input_tensor = K.concatenate([target_image,style_reference_image,combination_image], axis=0)

In [0]:
model = vgg19.VGG19(input_tensor = input_tensor, weights = 'imagenet', include_top = False)
print("Model Loaded")

W0623 21:26:18.961924 140607923914624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0623 21:26:18.967717 140607923914624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0623 21:26:19.010872 140607923914624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



80142336/80134624 [==============================] - 5s 0us/step


W0623 21:26:25.582750 140607923914624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0623 21:26:25.583976 140607923914624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



Model Loaded


<h3> Content loss <h3>

In [0]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

<h3> Style loss </h3>

In [0]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2,0,1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [0]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height*img_width
    return K.sum(K.square(C-S)) / (4 * (channels**2) * (size**2))

<h3> Total variation loss </h3>

In [0]:
def total_variation_loss(x):
    a = K.square(
    x[:,:img_height - 1, :img_width - 1, :] -  x[:,1:, :img_width - 1, :])
    
    b = K.square(
    x[:,:img_height - 1, :img_width - 1, :] -  x[:,1:, 1:, :])
    
    return K.sum(K.pow(a+b, 1.25))

<h3> Final loss </h3>

In [0]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [0]:
content_layer = 'block5_conv2'

high level feature to compute the loss for the content loss

In [0]:
style_layers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']

finding pattern from the style image from the lowest to highest convnet block

In [0]:
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

In [0]:
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :,:,:]
combination_features = layer_features[2, :, :, :]
loss+= content_weight * content_loss(target_image_features,combination_features)

W0623 21:26:46.394394 140607923914624 variables.py:2429] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


In [0]:
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :,:,:]
    combination_features = layer_features[2, :, :, :]
    s1 = style_loss(style_reference_features,combination_features)
    loss+= (style_weight/ len(style_layers)) * s1


In [0]:
loss += total_variation_weight * total_variation_loss(combination_image)

<h3> Setting up gradient descent process </h3>

In [0]:
grads = K.gradients(loss, combination_image)[0]

W0623 21:26:55.353736 140607923914624 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
fetch_loss_and_grads = K.function([combination_image], [loss,grads])

In [0]:
class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self,x):
        assert self.loss_value is None
        
        x=x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grads_values = grad_values
        
        return self.loss_value
    
    def grads(self,x):
        
        assert self.loss_value is not None
        grad_values = np.copy(self.grads_values)
        self.loss_value = None
        self.grads_values = None
        
        return grad_values

In [0]:
evaluator = Evaluator()

<h3> Style transfer loop </h3>

In [0]:
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time
#imageio.imwrite('filename.jpg', array)

In [0]:
result_prefix = 'my_result'
iterations = 30

In [0]:
x = preprocess_img(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time=time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height,img_width, 3))
    img = deprocess_img(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imageio.imwrite('/content/gdrive/My Drive/Dream/'+fname, img)
    print("Image saved as", fname)
    end_time = time.time()
    print('Iteration %d completed in %ds'%(i, end_time - start_time))

Start of iteration 0
Current loss value: 965760900.0
Image saved as my_result_at_iteration_0.png
Iteration 0 completed in 15s
Start of iteration 1
Current loss value: 467370700.0
Image saved as my_result_at_iteration_1.png
Iteration 1 completed in 8s
Start of iteration 2
Current loss value: 323383100.0
Image saved as my_result_at_iteration_2.png
Iteration 2 completed in 9s
Start of iteration 3
Current loss value: 257713550.0
Image saved as my_result_at_iteration_3.png
Iteration 3 completed in 9s
Start of iteration 4
Current loss value: 219491260.0
Image saved as my_result_at_iteration_4.png
Iteration 4 completed in 9s
Start of iteration 5
Current loss value: 194629090.0
Image saved as my_result_at_iteration_5.png
Iteration 5 completed in 9s
Start of iteration 6
Current loss value: 177063860.0
Image saved as my_result_at_iteration_6.png
Iteration 6 completed in 9s
Start of iteration 7
Current loss value: 160876750.0
Image saved as my_result_at_iteration_7.png
Iteration 7 completed in 9s